# Advanced python CoP

## Table of contents:
* Iter protocl
* Generators
* Contextlib

### Iter protocol

In [1]:
%%HTML
<iframe src='https://gfycat.com/ifr/YearlyWelcomeBlowfish' frameborder='0' scrolling='no' allowfullscreen width='640' height='1185'></iframe>

In [1]:
for value in [1, 2, 3]:
    print(value)

1
2
3


In [1]:
class Counter:
    pass

In [2]:
for value in Counter():
    print(value)

TypeError: 'Counter' object is not iterable

#### For loop under the hood

In [3]:
it = iter([1, 2, 3])
try:
    while True:
        value = next(it)
        print(value)
except StopIteration:
    pass

1
2
3


How to create iterable object? Implement `__iter__` that returns object implementing `__next__`

In [28]:
class Counter:
    def __init__(self, low, high):
        self.current = low
        self.high = high

    def __iter__(self):
        return self

    def __next__(self):
        if self.current > self.high:
            raise StopIteration
        else:
            self.current += 1
            return self.current - 1


for i in Counter(3, 8):
    print(i)

3
4
5
6
7
8


You don't have to return self in `__iter__`

In [7]:
class Counter:
    def __iter__(self):
        return iter([1, 2, 3])

for value in Counter():
    print(value)

1
2
3


Why is this information useful? Infinite or very large sequences.  
Problem: find first fibonaci sequence number which has a sum of digits greater than 100

In [11]:
from itertools import islice


class FibonacciIterator:
    def __iter__(self):
        self.a = 0
        self.b = 1
        return self

    def __next__(self):
        a, b = self.a, self.b
        self.a = b
        self.b = a + b
        return a


list(islice(FibonacciIterator(), 10))

[0, 1, 1, 2, 3, 5, 8, 13, 21, 34]

In [15]:
from typing import Iterable, Callable, Any

sum_digits = lambda n: sum(map(int, str(n)))

def find_first(iterable: Iterable, predicate: Callable[[Any], bool]):
    for x in iterable:
        if predicate(x):
            return x

find_first(FibonacciIterator(), lambda x: sum_digits(x) > 100)
# next(x for x in FibonaciIterator() if sum_digits(x) > 100)

218922995834555169026

### Generators

Generator is just like a container, but values are generated on the fly as you iterate.

In [16]:
generator = range(10000000)
big_list = list(generator)

from sys import getsizeof

print(getsizeof(generator))
print(getsizeof(big_list))

48
80000056


Generator comprehensions:

In [17]:
%%timeit
power_2 = [i**2 for i in range(10**6)]

292 ms ± 10 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [19]:
%%timeit
power_2_gen = (i**2 for i in range(10**6))

533 ns ± 13.6 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


It's waay faster because no `i**2` was computed :P. We only created a recipe for a sequence.

In [20]:
! pip3.8 install memory_profiler --user

In [21]:
%load_ext memory_profiler

In [22]:
%memit sum([i**2 for i in range(10**6)])

peak memory: 488.84 MiB, increment: 21.51 MiB


In [23]:
%memit sum(i**2 for i in range(10**6))

peak memory: 467.58 MiB, increment: 0.00 MiB


That's because with list you create the whole list and then start adding. With generator comprehension you ask the generator to generate next value and add it to the current sum. There's no need for a container.  

It's very useful when you might break at some point:

In [2]:
%%timeit
for i in range(10**6):
    if i == 2:
        break

361 ns ± 24.5 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [3]:
%%timeit
for i in list(range(10**6)):
    if i == 2:
        break

32.3 ms ± 1.16 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Yield

In [28]:
def some_generator():
    print("Starting")
    yield 1
    print("Let's come back to where we left off")
    yield 2
    print("Nope. No more yields")

In [29]:
gen = some_generator()

In [30]:
next(gen)

Starting


1

In [31]:
next(gen)

Let's come back to where we left off


2

In [32]:
next(gen)

Nope. No more yields


StopIteration: 

In [33]:
list(some_generator())

Starting
Let's come back to where we left off
Nope. No more yields


[1, 2]

In [34]:
gen = some_generator()

In [35]:
import inspect
inspect.getgeneratorstate(gen)

'GEN_CREATED'

In [36]:
next(gen)
inspect.getgeneratorstate(gen)

Starting


'GEN_SUSPENDED'

In [37]:
list(gen)
inspect.getgeneratorstate(gen)

Let's come back to where we left off
Nope. No more yields


'GEN_CLOSED'

In [38]:
def primitive_range(start: int, stop: int, step: int = 1):
    current = start
    while current < stop:
        yield current
        current += step


for i in primitive_range(0, 4):
    print(i)

0
1
2
3


#### The yield statement does 2 things. Freezes the current execution frame (function locals and next instruction to execute) and returns the value

If you come from C or C++ you might find it weird that stack (local variables) are not destructed when returing from function. In Cpython each function call creates new frame object on the heap :D. So python can manage the lifetime of function local variables dynamically.

In [40]:
import inspect
g = primitive_range(0,10)
inspect.getgeneratorlocals(g)

{'start': 0, 'stop': 10, 'step': 1}

In [41]:
next(g)

0

In [43]:
inspect.getgeneratorlocals(g)

{'start': 0, 'stop': 10, 'step': 1, 'current': 0}

In [45]:
next(g)

1

In [47]:
inspect.getgeneratorlocals(g)

{'start': 0, 'stop': 10, 'step': 1, 'current': 1}

How does python know where he stopped in generator? The instruction pointer

In [7]:
def simple_gen():
    x = 10
    yield x
    y = "abc"
    yield y

In [8]:
g = simple_gen()
g.gi_frame.f_lasti

-1

In [9]:
next(g)
g.gi_frame.f_lasti

6

In [10]:
import dis
dis.disco(g.gi_code, lasti=g.gi_frame.f_lasti)

  2           0 LOAD_CONST               1 (10)
              2 STORE_FAST               0 (x)

  3           4 LOAD_FAST                0 (x)
    -->       6 YIELD_VALUE
              8 POP_TOP

  4          10 LOAD_CONST               2 ('abc')
             12 STORE_FAST               1 (y)

  5          14 LOAD_FAST                1 (y)
             16 YIELD_VALUE
             18 POP_TOP
             20 LOAD_CONST               0 (None)
             22 RETURN_VALUE


This was python bytecode. And a topic for a separate CoP.

In [12]:
import dis
print(dis.code_info(simple_gen))

Name:              simple_gen
Filename:          <ipython-input-7-e977d2e2606f>
Argument count:    0
Positional-only arguments: 0
Kw-only arguments: 0
Number of locals:  2
Stack size:        1
Flags:             OPTIMIZED, NEWLOCALS, GENERATOR, NOFREE, 0x2000
Constants:
   0: None
   1: 10
   2: 'abc'
Variable names:
   0: x
   1: y


### Common generator pitfalls:

In [13]:
def infinite_power_2_gen():
    current = 2
    while True:
        yield current
        current *= 2

In [14]:
powers_of_2 = infinite_power_2_gen()
first_4 = powers_of_2[:4]

TypeError: 'generator' object is not subscriptable

There are a couple of ways to slice a generator

In [57]:
first_5_elements = []
for i in range(5):
    first_5_elements.append(next(powers_of_2))
first_5_elements

[2, 4, 8, 16, 32]

In [56]:
first_5_elements = [pair[0] for pair in zip(powers_of_2, range(5))]
first_5_elements

[64, 128, 256, 512, 1024]

In [59]:
from itertools import islice
first_4_powers_gen = islice(infinite_power_2_gen(), 4)
first_4_powers_gen

But obviously the islice is the best one.

Generators are one pass. There's no way to reuse a generator object that is already exhausted (that raised StopIteration)

In [61]:
list(first_4_powers_gen)

[]

With yield statement the FibonacciIterator implemented with iter protocol is much simplier:

In [66]:
def fib_generator():
    a, b = 0, 1
    while True:
        yield a
        a, b = b, a + b

In [67]:
list(islice(fib_generator(), 10))

[0, 1, 1, 2, 3, 5, 8, 13, 21, 34]

### Real use case - iterative xml parsing

In [6]:
from pathlib import Path
from typing import Generator
import xml.etree.ElementTree as ET


def iter_tags_from_xml_file(path: Path) -> Generator[ET.Element, None, None]:
    """
    Parses xml file incrementally to not bloat the ram on big xml files
    :param path: path to xml_file
    :return: generator iterating over xml tags
    """
    xml_iterator = iter(ET.iterparse(str(path), events=("start", "end")))
    _, root = next(xml_iterator)
    for event, element in xml_iterator:
        if event == "end":
            yield element
        # without clearing the root element the whole tree is still stored in ram, but created incrementally
        root.clear()

In [7]:
! du -hs ~/big.xml

19G	/home/rs/big.xml


In [8]:
tags = iter_tags_from_xml_file(Path("/home/rs/big.xml"))

In [31]:
keybox_tags = (entry for entry in tags if entry.tag == "Keybox")

It's great that up to this point the file hasn't been opened yet :D

In [75]:
%memit keybox_elements = sum(1 for x in keybox_tags)

peak memory: 468.92 MiB, increment: 0.00 MiB


We counted keybox tags in 17GB xml file with just a couple of KB.

### Data pipelines

We need some test data. Let's generate it using python

In [16]:
! pip3.8 install names --user

     |████████████████████████████████| 798kB 718kB/s eta 0:00:01
  Created wheel for names: filename=names-0.3.0-cp38-none-any.whl size=803690 sha256=964a90663fc1b9b1f186031d746fbd29dbc8a4ec96d2ec318b877e375aea310e
  Stored in directory: /home/rs/.cache/pip/wheels/f9/a5/e1/be3e0aaa6fa285575078fa2aafd9959b45bdbc8de8a6803aeb
Successfully built names


In [17]:
import names
import random

with open("student_grades.txt", "w") as file:
    for _ in range(25):
        print(f"{names.get_first_name()} {','.join(map(str,random.choices(range(2,6),k=15)))}", file=file)


In [18]:
!cat student_grades.txt

Krista 2,4,5,2,3,4,2,3,4,4,4,5,2,3,2
Gerald 3,5,5,2,4,2,2,2,5,3,5,4,2,2,5
Gilbert 2,3,4,3,4,4,2,2,4,5,2,3,3,3,3
Paul 4,3,3,4,3,3,3,2,2,2,5,5,5,3,5
Henry 2,5,4,3,5,4,3,4,5,4,4,5,4,4,5
Douglas 4,5,2,2,3,2,5,2,2,2,2,2,2,2,4
Michelle 4,3,4,2,3,4,5,3,2,2,3,4,3,5,4
Jessie 2,3,2,5,5,2,3,2,4,2,3,3,4,5,5
Shirley 4,4,3,2,3,3,3,4,2,5,5,4,4,2,5
Sean 4,5,3,3,4,3,4,5,3,5,5,4,4,4,2
Allen 5,3,2,3,2,3,3,3,4,3,5,5,3,3,4
Kara 3,4,3,5,3,4,4,2,5,3,4,3,4,4,2
Virginia 5,5,4,2,5,2,4,3,4,4,3,4,5,3,5
Marion 3,2,5,2,4,4,2,4,4,2,5,3,5,3,4
Latoya 3,2,2,2,5,3,4,5,2,2,3,5,3,4,4
Percy 4,5,2,5,5,2,4,5,5,5,4,4,5,3,4
Ching 2,3,5,5,4,4,3,3,4,5,5,5,4,3,3
Jane 4,2,3,4,4,3,2,2,3,2,2,3,2,4,2
William 3,5,4,3,5,4,5,5,2,5,3,5,5,3,3
Eleanor 5,2,5,5,2,5,3,2,4,4,2,2,4,5,4
Christopher 3,3,4,3,4,5,2,2,4,3,2,3,3,2,5
Todd 3,2,3,2,2,4,4,5,3,3,3,2,2,3,4
Richard 3,4,5,3,5,2,4,4,3,4,4,5,4,4,5
Michael 4,5,4,3,2,4,3,4,4,5,4,5,5,3,3
Nancy 3,5,5,4,3,2,2,4,5,5,5,5,5,5,2


Find the first student with grades mean less than 3.2. There's no need to find the one with the worst grades. Just find one  
Using lists won't scale. In case of a really big file you would run out of ram memory:

In [20]:
from pprint import pprint
from statistics import mean


def get_lines(filename: str):
    lines = []
    with open(filename) as file:
        for line in file:
            lines.append(line)
    return lines


def parse_lines(lines: list):
    students_with_grades = []
    for line in lines:
        student, grades_str = line.split()
        grades = [int(grade) for grade in grades_str.split(",")]
        students_with_grades.append((student, grades))
    return students_with_grades


def get_students_with_means(students: list):
    student_with_means = []
    for student, grades in students:
        student_with_means.append((student, mean(grades)))
    return student_with_means


lines = get_lines("student_grades.txt")
students_with_grades = parse_lines(lines)
student_with_means = get_students_with_means(students_with_grades)
student_with_means

[('Krista', 3.2666666666666666),
 ('Gerald', 3.4),
 ('Gilbert', 3.1333333333333333),
 ('Paul', 3.466666666666667),
 ('Henry', 4.066666666666666),
 ('Douglas', 2.7333333333333334),
 ('Michelle', 3.4),
 ('Jessie', 3.3333333333333335),
 ('Shirley', 3.533333333333333),
 ('Sean', 3.8666666666666667),
 ('Allen', 3.4),
 ('Kara', 3.533333333333333),
 ('Virginia', 3.8666666666666667),
 ('Marion', 3.466666666666667),
 ('Latoya', 3.2666666666666666),
 ('Percy', 4.133333333333334),
 ('Ching', 3.8666666666666667),
 ('Jane', 2.8),
 ('William', 4),
 ('Eleanor', 3.6),
 ('Christopher', 3.2),
 ('Todd', 3),
 ('Richard', 3.933333333333333),
 ('Michael', 3.8666666666666667),
 ('Nancy', 4)]

In [22]:
next(((student, mean) for student, mean in student_with_means if mean < 3.1))

('Douglas', 2.7333333333333334)

But this will scale. I've put the corresponding list-generator functions next to each other, so you can how simple was the transition from lists to generators:

In [23]:
from pprint import pprint
from statistics import mean

def get_lines(filename: str):
    lines = []
    with open(filename) as file:
        for line in file:
            lines.append(line)
    return lines


def get_lines_gen(filename: str):
    with open(filename) as file:
        for line in file:
            yield line


def parse_lines_gen(lines):
    for line in lines:
        student, grades_str = line.split()
        grades = [int(grade) for grade in grades_str.split(",")]
        yield (student, grades)
        
def parse_lines(lines):
    students_with_grades = []
    for line in lines:
        student, grades_str = line.split()
        grades = [int(grade) for grade in grades_str.split(",")]
        students_with_grades.append((student, grades))
    return students_with_grades


def get_students_with_means(students):
    student_with_means = []
    for student, grades in students:
        student_with_means.append((student, mean(grades)))
    return student_with_means

def get_students_with_means_gen(students):
    for student, grades in students:
        yield (student, mean(grades))


In [24]:
lines = get_lines_gen("student_grades.txt")
students_with_grades = parse_lines_gen(lines)
student_with_means = get_students_with_means_gen(students_with_grades)

It's great that up to this point no line has been read from file.

In [25]:
list(student_with_means)

[('Krista', 3.2666666666666666),
 ('Gerald', 3.4),
 ('Gilbert', 3.1333333333333333),
 ('Paul', 3.466666666666667),
 ('Henry', 4.066666666666666),
 ('Douglas', 2.7333333333333334),
 ('Michelle', 3.4),
 ('Jessie', 3.3333333333333335),
 ('Shirley', 3.533333333333333),
 ('Sean', 3.8666666666666667),
 ('Allen', 3.4),
 ('Kara', 3.533333333333333),
 ('Virginia', 3.8666666666666667),
 ('Marion', 3.466666666666667),
 ('Latoya', 3.2666666666666666),
 ('Percy', 4.133333333333334),
 ('Ching', 3.8666666666666667),
 ('Jane', 2.8),
 ('William', 4),
 ('Eleanor', 3.6),
 ('Christopher', 3.2),
 ('Todd', 3),
 ('Richard', 3.933333333333333),
 ('Michael', 3.8666666666666667),
 ('Nancy', 4)]

In [35]:
lines = get_lines_gen("student_grades.txt")
students_with_grades = parse_lines_gen(lines)
student_with_means = get_students_with_means_gen(students_with_grades)
next(((student, mean) for student, mean in student_with_means if mean < 3.1))

('Douglas', 2.7333333333333334)

With this transition we came from:
* read all lines, store them in a list
* parse each lines, store parsed lines in a list
* count average grades for each student, store the results in a list
* iterate through list with averages to find the student that should be expelled

to:  

* read first line
* parse this line
* count the average
* check if it's the student that should be expelled
* repeat until we found the student

The second version is better because:
* it scales for a really big lists that cannot be loaded to ram memory
* it can be easilly parallelised as a standard consumer producer problem
* it is faster because you read and parse just enogh lines to find the correct student


It can also be done with pure generator comprehensions in just a couple of lines. I love python one-liners:

In [36]:
lines = (line for line in open("student_grades.txt"))
splitted_lines = (line.split() for line in lines)
students_with_grades = ((student, [int(grade) for grade in grades_str.split(",")]) for student, grades_str in splitted_lines)
students_with_means = ((student, mean(grades)) for student, grades in students_with_grades)
next(((student, mean) for student, mean in students_with_means if mean < 3.1))

('Douglas', 2.7333333333333334)

### Other generator methods: send, throw, close

In [42]:
def adjustable_counter():
    current = 0
    while True:
        next_val = yield current
        if next_val is not None:
            current = next_val
        current += 1


c = adjustable_counter()

In [43]:
next(c)

0

In [44]:
next(c)

1

In [45]:
next(c)

2

With send() you can send a value to the runnig generator. next() is equivalent to send(None)

In [46]:
c.send(-100) # Spoiler alert - that's how coroutines communitate

-99

In [47]:
next(c)

-98

throw() raises the exception inside the generator:

In [48]:
c.throw(RuntimeError("Sorry"))

RuntimeError: Sorry

Close raises GeneratorExit inside the generator. This cleans up the generator state.

In [52]:
c.close()

In [53]:
import inspect
inspect.getgeneratorstate(c)

'GEN_CLOSED'

### Context managers - RAII in Python

In [126]:
with open("irrelevant.txt","w") as file:
    file.write("raii")

Is better than:

In [127]:
file = open("irrelevant.txt","w")
try:
    file.write("raii")
finally:
    file.close()

In [128]:
from threading import Lock

lock = Lock()
x = 10

In [129]:
lock.acquire()
x += 1
lock.release()

In [130]:
with lock:
    x += 1
with lock:
    x += 1

In [54]:
%%HTML
<blockquote class="reddit-card" data-card-created="1570178622"><a href="https://www.reddit.com/r/ProgrammerHumor/comments/bfr1xc/i_love_python_but/">I love Python, but...</a> from <a href="http://www.reddit.com/r/ProgrammerHumor">r/ProgrammerHumor</a></blockquote>
<script async src="//embed.redditmedia.com/widgets/platform.js" charset="UTF-8"></script>

#### How does this work under the hood?

In [133]:
class File:
    def __init__(self, name: str, mode: str = "r"):
        self.name = name
        self.mode = mode
        self.file_handle = None

    def __enter__(self):
        self.file_handle = open(self.name, self.mode)
        return self.file_handle

    def __exit__(self, exc_type, exc_val, exc_tb):
        print("__exit__ called")
        if self.file_handle:
            self.file_handle.close()


with File("irrelevant.txt", "r") as f:
    10 / 0

__exit__ called


ZeroDivisionError: division by zero

### contextlib

@contextmanager - a shortcut for creating contextmanagers  
The code up to the first yield statement is executed in `__enter__` and the rest is executed in `__exit__`

In [59]:
from contextlib import contextmanager


@contextmanager
def File(name: str, mode: str = "r"):
    file_handle = None
    try:
        file_handle = open(name, mode)
        yield file_handle
    finally:
        if file_handle:
            print("closing")
            file_handle.close()

In [60]:
with File("irrelevant.txt", "w") as f:
    pass

closing


In [62]:
with File("irrelevant.txt", "w") as f:
    10 / 0

closing


ZeroDivisionError: division by zero

In [63]:
with File("3.txt", "r") as f:
    10 / 0

FileNotFoundError: [Errno 2] No such file or directory: '3.txt'

In [138]:
import sys
import datetime
from typing import Generator
from typing.io import TextIO
from contextlib import contextmanager



@contextmanager
def execution_time_printed(file: TextIO = sys.stdout) -> Generator[None, None, None]:
    start = datetime.datetime.now()
    yield
    print("Execution time:", datetime.datetime.now() - start, file=file)

In [139]:
with execution_time_printed():
    print("inside")
    import time
    time.sleep(0.5)
print("outside")

inside
Execution time: 0:00:00.500736
outside


### Reentrant contextmanagers

In [140]:
file = open("irrelevant.txt","w")
with file:
    file.write("a")


with file:
    file.write("a")

ValueError: I/O operation on closed file.

SQL transaction is implemented as contextmanager:

In [141]:
import sqlite3
db = sqlite3.connect(":memory:")
with db:
    db.execute("")
    db.execute("")

In [142]:
with db:
    db.execute("")
    db.execute("")

### Btw threading pool context manager

In [65]:
from concurrent.futures import ThreadPoolExecutor

with ThreadPoolExecutor(max_workers=1) as executor:
    future = executor.submit(pow, 323, 1235)

print(future.result())

7330187419711662525292446729952277967765833457839424373869116780517420149076198183898894905236339668910883614705749593461204605714622607490668683222808958179218191780471063516227460897872227090014097232715359988867409006240081206335670819048146328123935337644631341482039026277834498173918554303355637321060412263715606736911839847081166018722332660742474936263046482602637679145832497919840537694829188335160914131310111239449199642739655793719812086149415859534959085359215402107080568853413877372159233452025442228651418507639010743174496936173262981681095359156359401217962539764203947129055258900852300663811552683018727645219707243611502505332407412509113706415784954450373499498470564461122438759199401776852200401606640428677779377601753155808428527723343610817938497649317098807976560094452142608380040084257453294862872175833371967399756248792099979580669182692895291737594003884626270278670234491588018888596571060169103937561057223083861024046092685665503687387204434227037124219374130642

In [6]:
from concurrent.futures import Executor, ProcessPoolExecutor

with ProcessPoolExecutor(max_workers=8) as executor:
    executor: Executor
    powers = list(executor.map(pow, range(10 ** 4), range(10 ** 4)))
powers[:100]

[1,
 1,
 4,
 27,
 256,
 3125,
 46656,
 823543,
 16777216,
 387420489,
 10000000000,
 285311670611,
 8916100448256,
 302875106592253,
 11112006825558016,
 437893890380859375,
 18446744073709551616,
 827240261886336764177,
 39346408075296537575424,
 1978419655660313589123979,
 104857600000000000000000000,
 5842587018385982521381124421,
 341427877364219557396646723584,
 20880467999847912034355032910567,
 1333735776850284124449081472843776,
 88817841970012523233890533447265625,
 6156119580207157310796674288400203776,
 443426488243037769948249630619149892803,
 33145523113253374862572728253364605812736,
 2567686153161211134561828214731016126483469,
 205891132094649000000000000000000000000000000,
 17069174130723235958610643029059314756044734431,
 1461501637330902918203684832716283019655932542976,
 129110040087761027839616029934664535539337183380513,
 11756638905368616011414050501310355554617941909569536,
 1102507499354148695951786433413508348166942596435546875,
 106387358923716524807713475752